In [5]:
import numpy as np
import tensorflow as tf
import scipy as sp

#define data structures
image_data = []
imu_data = []
state_data = []

#camera intrinsics
fx = tf.Variable(500.0)
fy = tf.Variable(500.0)
cx = tf.Variable(300.0)
cy = tf.Variable(200.0)
K = tf.sparse.SparseTensor(indices=[[0,0],[0,2],[1,1],[1,2],[2,2]],values=[fx,cx,fy,cy,1.0],dense_shape=[3,3])
#imu intrinsics
#camera_imu_extend
r = tf.Variable(0.0)
p = tf.Variable(0.0)
y = tf.Variable(0.0)
t = tf.Variable([[0.0],[0.0],[0.0]])
C_r = tf.sparse.SparseTensor(indices=[[0,0],[1,1],[1,2],[2,1],[2,2]],values=[1.0,tf.math.cos(r),-tf.math.sin(r),tf.math.sin(r),tf.math.cos(r)],dense_shape=[3,3])
C_p = tf.sparse.SparseTensor(indices=[[0,0],[0,2],[1,1],[2,0],[2,2]],values=[tf.math.cos(p),tf.math.sin(p),1.0,-tf.math.sin(p),tf.math.cos(p)],dense_shape=[3,3])
C_y = tf.sparse.SparseTensor(indices=[[0,0],[0,1],[1,0],[1,1],[2,2]],values=[tf.math.cos(y),-tf.math.sin(y),tf.math.sin(y),tf.math.cos(y),1.0],dense_shape=[3,3])
T_ir = tf.matmul(tf.matmul(tf.sparse_tensor_to_dense(C_y),tf.sparse_tensor_to_dense(C_p)),tf.sparse_tensor_to_dense(C_r))
T_ir = tf.concat([T_ir,t],1)
T_ir = tf.concat([T_ir,tf.constant([[0.0,0.0,0.0,1.0]])],0)
r_c = tf.Variable(1.57)
p_c = tf.Variable(0.0)
y_c = tf.Variable(0.0)
C_rc = tf.sparse.SparseTensor(indices=[[0,0],[1,1],[1,2],[2,1],[2,2]],values=[1.0,tf.math.cos(r_c),-tf.math.sin(r_c),tf.math.sin(r_c),tf.math.cos(r_c)],dense_shape=[3,3])
C_pc = tf.sparse.SparseTensor(indices=[[0,0],[0,2],[1,1],[2,0],[2,2]],values=[tf.math.cos(p_c),tf.math.sin(p_c),1.0,-tf.math.sin(p_c),tf.math.cos(p_c)],dense_shape=[3,3])
C_yc = tf.sparse.SparseTensor(indices=[[0,0],[0,1],[1,0],[1,1],[2,2]],values=[tf.math.cos(y_c),-tf.math.sin(y_c),tf.math.sin(y_c),tf.math.cos(y_c),1.0],dense_shape=[3,3])
T_ci = tf.matmul(tf.matmul(tf.sparse_tensor_to_dense(C_yc),tf.sparse_tensor_to_dense(C_pc)),tf.sparse_tensor_to_dense(C_rc))
T_ci = tf.concat([T_ci,t],1)
T_ci = tf.concat([T_ci,tf.constant([[0.0,0.0,0.0,1.0]])],0)
K_d = tf.sparse_tensor_to_dense(K)
sess = tf.compat.v1.Session()
init_op = tf.compat.v1.global_variables_initializer()
sess.run(init_op)
print(sess.run(C_r))
print(sess.run(C_p))
print(sess.run(T_ci))
print(sess.run(T_ir))
print(sess.run(K_d))


SparseTensorValue(indices=array([[0, 0],
       [1, 1],
       [1, 2],
       [2, 1],
       [2, 2]]), values=array([ 1.,  1., -0.,  0.,  1.], dtype=float32), dense_shape=array([3, 3]))
SparseTensorValue(indices=array([[0, 0],
       [0, 2],
       [1, 1],
       [2, 0],
       [2, 2]]), values=array([ 1.,  0.,  1., -0.,  1.], dtype=float32), dense_shape=array([3, 3]))
[[ 1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  7.962743e-04 -9.999997e-01  0.000000e+00]
 [ 0.000000e+00  9.999997e-01  7.962743e-04  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[500.   0. 300.]
 [  0. 500. 200.]
 [  0.   0.   1.]]


In [42]:
# place holder for measurements and states
#time
T = tf.compat.v1.placeholder(tf.int32,shape = [])
J = tf.compat.v1.placeholder(tf.int32,shape = [])
M = tf.compat.v1.placeholder(tf.int32,shape = [])
#true states:
true_a = tf.compat.v1.placeholder(tf.float32,shape = (None,3))
true_w =  tf.compat.v1.placeholder(tf.float32,shape = (None,3))
#true transformations:
true_C_rw = tf.compat.v1.placeholder(tf.float32,shape = (None,3,3))
true_T_rw = tf.compat.v1.placeholder(tf.float32,shape = (None,4,4))
#gravity
g_w = tf.compat.v1.placeholder(tf.float32,shape = [])
#landmark
p_w = tf.compat.v1.placeholder(tf.float32,shape = (None,4))
#bias (state)
b_a = tf.Variable([], validate_shape=False)
b_w = tf.Variable([], validate_shape=False)
#imu noise
R_a = tf.placeholder(tf.float32,shape=(3,3))
R_w = tf.placeholder(tf.float32,shape =(3,3))
#measurements
y_m = tf.compat.v1.placeholder(tf.float32,shape = (None,None,2))
a_m = tf.compat.v1.placeholder(tf.float32,shape = (None,3))
w_m = tf.compat.v1.placeholder(tf.float32,shape = (None,3))
##estimations
#acceleration
flat_true_C_i = tf.reshape(true_C_iw, (-1,3))
flat_true_a_m = tf.reshape(tf.matmul(flat_true_C_i,tf.transpose(true_a-g_w)),(-1,3))+b_a
a_m_error = flat_true_a_m-a_m
a_m_cost = tf.linalg.trace(tf.matmul(tf.matmul(a_m_error,R_a),tf.transpose(a_m_error)))
#angular velocity
flat_true_w_m = tf.reshape(tf.matmul(flat_true_C_i,tf.transpose(true_w)),(-1,3))+b_w
w_m_error = flat_true_w_m-w_m
w_m_cost = tf.linalg.trace(tf.matmul(tf.matmul(w_m_error,R_w),tf.transpose(w_m_error)))
#reprojection
p_i = tf.matmul(true_T_iw,tf.transpose(p_w))
p_c = tf.matmul(tf.transpose(p_i,[0,2,1]),tf.transpose(T_ci))
T_c_T = tf.concat([tf.eye(3,dtype=tf.float32),tf.zeros([1,3],tf.float32)],0)
true_y_h= tf.matmul(tf.matmul(p_c,T_c_T),tf.transpose(K_d))
true_y_m = tf.divide(true_y_h[:,:,0:2],true_y_h[:,:,2])
print(true_y_m)
y_m_error = 

Tensor("truediv_11:0", shape=(?, ?, 2), dtype=float32)


In [2]:
import cv2
import apriltag

img = cv2.imread('april.jpeg',0)
print(img)
detector = apriltag.Detector()
result = detector.detect(img)
print(result)
cv2.imshow('image',img)

[[255 255 255 ... 255 246 249]
 [255 255 255 ... 247 255 255]
 [255 255 255 ...  16 148 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
[]


In [19]:
import numpy as np
import cv2
import glob
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,6), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

In [25]:
print(images)
print(np.max(np.asarray(imgpoints).reshape((-1,3)),axis=0))

['index1.jpg', 'index1 (copy).jpg', 'calib_pattern.jpg', 'index.jpg']
[179.38551 179.38715 179.39127]


In [28]:
img_flat = np.reshape(np.asarray(imgpoints),(-1,3))
max_x = np.max(img_flat,axis = 0)[0]
max_y = np.max(img_flat,axis = 0)[1]
min_x = np.min(img_flat,axis = 0)[0]
min_y = np.min(img_flat,axis = 0)[1]
print(max_x)
print(max_y)
print(img_flat)

179.38551
179.38715
[[155.50456   35.45695  155.50603 ]
 [ 59.492615 155.50597   83.41682 ]
 [155.50362  107.426    155.50261 ]
 [131.42165  155.50293  155.36176 ]
 [155.50533  179.38715  131.51321 ]
 [ 35.453564 131.51111   59.490196]
 [131.5109    83.41406  131.51549 ]
 [107.42185  131.50717  131.41592 ]
 [131.51888  155.3596   131.51335 ]
 [179.37946  107.49776   35.457188]
 [107.49963   59.494373 107.497284]
 [ 83.41861  107.50053  107.433495]
 [107.49707  131.42744  107.49827 ]
 [155.36656  107.49934  179.39127 ]
 [ 83.483826  35.45153   83.48429 ]
 [ 59.490486  83.483635  83.41224 ]
 [ 83.48761  107.42182   83.484665]
 [131.41637   83.49198  155.36066 ]
 [ 83.48788  179.38045   59.501244]
 [ 35.452023  59.500084  59.49359 ]
 [ 59.50057   83.41248   59.500355]
 [107.43131   59.499207 131.4193  ]
 [ 59.498898 155.36526   59.50042 ]
 [179.38551   35.479362  35.44871 ]
 [ 35.483818  59.49008   35.48099 ]
 [ 83.41172   35.48924  107.422676]
 [ 35.482693 131.41776   35.492374]
 [155.36

In [36]:
list1 = np.arange(0,len(imgpoints),1)
mtx = np.zeros((3,3))
for i in range(3):
    cur_data = list1
    print(list1)
    if len(imgpoints)>20:
        cur_data = sample(list,20)
    cur_obj = list(objpoints[i] for i in cur_data)
    cur_img = list(imgpoints[i] for i in cur_data)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(cur_obj, cur_img, gray.shape[::-1],cameraMatrix=mtx, criteria=criteria_cal)

[0 1]


NameError: name 'criteria_cal' is not defined